# Cluster Network Creation

Perform computationally expensive network analysis on a computing cluster. Results are pushed to a data repository on GitHub.

In [25]:
# to allow relative imports
import os
import sys

module = os.path.abspath(os.path.join('..'))
lib = os.path.abspath(os.path.join(module, os.pardir))
if lib not in sys.path:
    sys.path.append(lib)

# experiment
from epyc import Lab, JSONLabNotebook, RepeatedExperiment, ParallelLab
from lib.experiments.utils.metrics import MetricExperiment

# generators
from epydemic import PLCNetwork
from lib.model.network.mobility_network import MNGeneratorFromNetworkData as MNG
from lib.model.network.distanced_network import DNGenerator as DNG

# distributions
from lib.model.distributions import discrete_trunc_normal, discrete_trunc_exponential, num_contact_dist

# file utils
from lib.experiments.utils.data_repo_api import DataRepoAPI
from lib.experiments.utils.network_data_utils import load_network_data_from_files

# other utils
from functools import partial
import numpy as np
from mpmath import polylog
from datetime import datetime as dt

# multiprocessing
from joblib import Parallel, delayed
from multiprocessing import cpu_count

In [2]:
# Some global parameters

N = 10000
CUTOFF = 40
MOBILITY_EXPONENT = 2
n_exp = 10

# Experiment result output dir
output_dir = 'experiment_results'
if not output_dir in os.listdir():
    os.mkdir(output_dir)

# Helper functions
def run_network_lab_experiment(generator, params, file_name, description, n=n_exp,
                               output_dir=output_dir, push_to_repo=True):
    """
    Run a MetricExperiment repeatadly using a network generator.
    """

    print(f"%s - Running {file_name}" % dt.now())
    
    file = os.path.join(output_dir, file_name)
    
    nb = JSONLabNotebook(file, create=True, description=description)
    lab = ParallelLab(notebook=nb, cores=min(cpu_count()-1, n))
    
    e = MetricExperiment(generator)

    # pass params to the lab
    for k, v in params.items():
        lab[k] = v

    lab.runExperiment(RepeatedExperiment(e, n))
    
    print("%s - Done." % dt.now())

    if push_to_repo:
        DataRepoAPI.update_or_create(
            file_name=file_name,
            file_path=output_dir,
            repo_path='network-metrics'
        )

    return lab.results()

def plc_mean(exponent, cutoff):
    """
    Calculate theoretical mean of a power law with cutoff distribution from
    the exponent and the cutoff.
    """
    nom = polylog(exponent-1, np.exp(-1 / cutoff))
    denom = polylog(exponent, np.exp(-1 / cutoff))
    return nom / denom

def estimate_exponent_for_mean(mean, cutoff, tolerance):
    """
    Given the mean and cutoff of a power law with cutoff distribution,
    estimate the corresponding exponent with a given tolerance.
    """
    grid = np.arange(1, 5, 0.01)
    diffs = []
    for i in range(len(grid)):
        diff = abs(mean - plc_mean(grid[i], cutoff))
        if diff < tolerance:
            return grid[i]
        diffs.append(diff)
    
    return grid[diffs.index(min(diffs))]

### Mobility network

In [3]:
# Load the network data

file_names = dict(
    demographics='demographics.pkl',
    comb_pre='comb_counts_pre.pkl',
    comb_post='comb_counts_post.pkl',
    trip_pre='trip_counts_pre.pkl',
    trip_post='trip_counts_post.pkl',
)

network_data = load_network_data_from_files(file_names)

network_data_pre = network_data['pre']
network_data_post = network_data['post']

In [4]:
# Define the generator parameters
params_mobility_pre = dict()
params_mobility_pre[MNG.N] = N
params_mobility_pre[MNG.EXPONENT] = MOBILITY_EXPONENT
params_mobility_pre[MNG.CUTOFF] = CUTOFF
params_mobility_pre[MNG.MULTIPLIER] = False

params_mobility_post = dict()
params_mobility_post[MNG.N] = N
params_mobility_post[MNG.EXPONENT] = MOBILITY_EXPONENT
params_mobility_post[MNG.CUTOFF] = CUTOFF
params_mobility_post[MNG.MULTIPLIER] = True

In [5]:
# Generator instances
mng_pre = MNG(network_data=network_data_pre)
mng_post = MNG(network_data=network_data_post)

In [6]:
# Run experiment
mng_pre_results = run_network_lab_experiment(
    mng_pre, 
    params_mobility_pre, 
    'mobility_pre.json', 
    'Network metrics of Mobility pre'
)


mng_post_results = run_network_lab_experiment(
    mng_post, 
    params_mobility_post, 
    'mobility_post.json', 
    'Network metrics of Mobility post'
)

2021-07-15 19:49:46.492886 - Running mobility_pre.json
2021-07-15 19:55:53.757167 - Done.
2021-07-15 19:55:53.760562 - Running mobility_post.json
2021-07-15 20:00:24.468130 - Done.


In [7]:
def extract_mean_deg(results):
    deg_lis = [x['results']['degrees'] for x in results]
    return np.mean([i for s in deg_lis for i in s])

# mob_deg_pre = extract_mean_deg(mng_pre_results)
# mob_deg_post = extract_mean_deg(mng_post_results)

In [ ]:
# get from repo...
distanced_pre = DataRepoAPI.get_json_file('network-metrics/distanced_pre.json')
distanced_post = DataRepoAPI.get_json_file('network-metrics/distanced_post.json')

_results_pre = distanced_pre['resultsets']['epyc.resultset.default']['results']
_results_post = distanced_post['resultsets']['epyc.resultset.default']['results']

mob_deg_pre = extract_mean_deg(_results_pre)
mob_deg_post = extract_mean_deg(_results_post)

### PLC network

We needed the results of the mng for calculating the exponent of the other experiments.. but the rest we can run in parallel to save some time...

In [8]:
# List to store all the metric experiment runners
mtrcs = list()

In [9]:
# Estimate the exponent for the PLC to achieve same degree
#  as the mobility networks

plc_expo_pre = estimate_exponent_for_mean(mob_deg_pre, CUTOFF, 0.01)
plc_expo_post = estimate_exponent_for_mean(mob_deg_post, CUTOFF, 0.01)

In [10]:
# Define the generator parameters
params_plc_pre = dict()
params_plc_pre[PLCNetwork.N] = N
params_plc_pre[PLCNetwork.EXPONENT] = plc_expo_pre
params_plc_pre[PLCNetwork.CUTOFF] = CUTOFF

params_plc_post = dict()
params_plc_post[PLCNetwork.N] = N
params_plc_post[PLCNetwork.EXPONENT] = plc_expo_post
params_plc_post[PLCNetwork.CUTOFF] = CUTOFF

In [11]:
plc_pre = PLCNetwork()
plc_post = PLCNetwork()

In [12]:
mtrcs.append(lambda: run_network_lab_experiment(
    plc_pre, 
    params_plc_pre, 
    'plc_pre.json', 
    'Network metrics of PLC pre'
))

mtrcs.append(lambda: run_network_lab_experiment(
    plc_post, 
    params_plc_post, 
    'plc_post.json', 
    'Network metrics of PLC post'
))

### Distanced network

In [13]:
household_size_dist = partial(discrete_trunc_normal, mu=4.5, std=2)

params_distanced_pre = dict()
params_distanced_pre[DNG.N] = N

params_distanced_post = dict()
params_distanced_post[DNG.N] = N

In [14]:
distanced_pre = DNG(
    household_size_dist=household_size_dist,
    num_contact_dist=num_contact_dist,
    num_outside_edge_dist=partial(discrete_trunc_exponential, exponent=mob_deg_pre)
)

distanced_post = DNG(
    household_size_dist=household_size_dist,
    num_contact_dist=num_contact_dist,
    num_outside_edge_dist=partial(discrete_trunc_exponential, exponent=mob_deg_post)
)

In [15]:
mtrcs.append(lambda: run_network_lab_experiment(
    distanced_pre, 
    params_distanced_pre, 
    'distanced_pre.json', 
    'Network metrics of PLC pre'
))

mtrcs.append(lambda: run_network_lab_experiment(
    distanced_post, 
    params_distanced_post, 
    'distanced_post.json', 
    'Network metrics of PLC post'
))

In [22]:
# %%time
# num_cores = min(len(mtrcs), cpu_count()-1)
# with Parallel(n_jobs=num_cores) as processes:
#     procs = processes(delayed(e)() for e in mtrcs)

for mtrc in mtrcs:
    print("Start time: %s" % dt.now())
    mtrc()
    print("End time: %s" % dt.now())

Done
Done
Done
Done
CPU times: user 25.9 ms, sys: 5.08 ms, total: 31 ms
Wall time: 3min 42s


# Epidemic models

In [254]:
# List to store all the experiment runners
expms = list()

In [255]:
# Import models etc
from epydemic import SEIR
from lib.model.compartmental_model.seir import MonitoredSEIR
from lib.model.compartmental_model.seir import SEIRWithQuarantine as SEIRQ
from lib.model.compartmental_model.seir import MonitoredSEIRWithQuarantine as MonitoredSEIRQ
from lib.model.compartmental_model.seivr import SEIVR, MonitoredSEIVR
from lib.model.compartmental_model.seivr import SEIVRWithQuarantine as SEIVRQ
from lib.model.compartmental_model.seivr import MonitoredSEIVRWithQuarantine as MonitoredSEIVRQ

from epydemic import StochasticDynamics, Monitor

from lib.model.distributions import PowerLawCutoffDist as PLCD

In [256]:
# Helper functions

def calc_p_infect(r0, p_remove, k_mean, k_var):
    """
    Calc p_infect.
    :param r0: basic reproduction number
    :param p_remove: probability of removal
    :param k_mean: Mean of node degree distribution
    :param k_var: Variance of node degree distribution
    :return: p_infect
    """
    return float(p_remove * r0 * (k_mean / (k_var - k_mean)))


def run_stochastic_dynamics_lab(model, generator, params, 
                                file_name, description, n=n_exp,
                                output_dir=output_dir, push_to_repo=True,
                                T_max=300):
    """
    Run a StochasticDynamics experiment repeatadly in a lab.
    """

    file = os.path.join(output_dir, file_name)
    
    nb = JSONLabNotebook(file, create=True, description=description)
    lab = ParallelLab(notebook=nb, cores=min(cpu_count()-1, n))
    
    e = StochasticDynamics(model, generator)
    e.process().setMaximumTime(T_max)

    # pass params to the lab
    for k, v in params.items():
        lab[k] = v

    lab.runExperiment(RepeatedExperiment(e, n))
    
    if push_to_repo:
        DataRepoAPI.update_or_create(
            file_name=file_name,
            file_path=output_dir,
            repo_path='simulations'
        )
    
    return lab.results()

## Parameters

We will fix as many COVID-19 specific parameters as we can and leave only policy dependent parameters open for modifications.

We set the following parameters

| Variable | Value | Description | Reference |
| :-- | --: | :-- | :-- |
| R0 | $2.85$ | Basic Reproduction Number | 1) |
| P_INFECT | ... | P. of infection | ... |
| P_REMOVE | $0.1$ | P. of removal | 2) |
| P_SYMPTOPMS | $0.2$ | P. of becoming symptomatic | 3) |
| P_EXPOSED | $0.01$ | P. of initial infection | ... |
| RRR | $0.95$, $0.67$ | Vaccine efficacy | 4), 5), 6) |
| P_VACCINATED | $10^{-3}$ to $10^{-2}$ by $10^{-3}$| P. getting vaccinated | ... |
| P_VACCINATED_INITIAL | $0$ to $1$ by $0.1$ | P. of initial vaccination | ... |
| P_QUARANTINE | $0$ to $1$ by $0.1$ | P. of rewiring | ... |


1) https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0249271

2) https://www.thelancet.com/journals/lanmic/article/PIIS2666-5247(20)30172-5/fulltext

3) https://doi.org/10.3389/fpubh.2020.598547

4) https://www.nejm.org/doi/full/10.1056/NEJMoa2034577

5) https://www.nejm.org/doi/full/10.1056/nejmoa2035389

6) https://www.sciencedirect.com/science/article/pii/S0140673620326611

In [257]:
# Fixed
R0 = 2.85
_P_INFECT = None  # to calculate later 
P_REMOVE = 0.1
P_SYMPTOMS = 0.2
P_EXPOSED = 0.01

_RRR = np.arange(0.5, 1, 0.15)

# Free
_P_VACCINATED = np.arange(0.001, 0.011, 0.003)
_P_VACCINATED_INITAL = np.arange(0, 1.1, 0.2)
_P_QUARANTINE = np.arange(0, 1.1, 0.25)

In [258]:
# Fix parameters as far as possible

params_seir = dict()
params_seir[SEIR.P_SYMPTOMS] = P_SYMPTOMS
params_seir[SEIR.P_REMOVE] = P_REMOVE
params_seir[SEIR.P_EXPOSED] = P_EXPOSED

params_seivr = dict()
params_seivr[SEIVR.P_SYMPTOMS] = P_SYMPTOMS
params_seivr[SEIVR.P_REMOVE] = P_REMOVE
params_seivr[SEIVR.P_EXPOSED] = P_EXPOSED

params_monitor = dict()
params_monitor[Monitor.DELTA] = 10

In [259]:
# Calculate P_INFECT

# Mobility
plcd = PLCD(MOBILITY_EXPONENT, CUTOFF)
P_INFECT_MOBILITY = calc_p_infect(R0, P_REMOVE, plcd.mean, plcd.var)

# PLC
plcd = PLCD(plc_expo_pre, CUTOFF)
P_INFECT_PLC = calc_p_infect(R0, P_REMOVE, plcd.mean, plcd.var)

# Distanced
mean = mob_deg_pre
var = mean ** 2
P_INFECT_DISTANCED = calc_p_infect(R0, P_REMOVE, mean, var)

# clean up
del mean, var, plcd

### SEIR & SEIR_Q

#### SEIR on Mobility

In [146]:
# params
params_pre1   = {**params_mobility_pre, **params_seir, **params_monitor}
params_pre1[SEIR.P_INFECT_SYMPTOMATIC] = P_INFECT_MOBILITY
params_pre1[SEIR.P_INFECT_ASYMPTOMATIC] = P_INFECT_MOBILITY / 2

params_post1 = {**params_mobility_post, **params_seir, **params_monitor}
params_post1[SEIR.P_INFECT_SYMPTOMATIC] = P_INFECT_MOBILITY
params_post1[SEIR.P_INFECT_ASYMPTOMATIC] = P_INFECT_MOBILITY / 2

In [147]:
expms.append(lambda: run_stochastic_dynamics_lab(
    MonitoredSEIR(),
    mng_pre, 
    params_pre1, 
    'seir_mobility_pre.json',
    'SEIR simulation with Mobility Network (Pre)',
))

expms.append(lambda: run_stochastic_dynamics_lab(
    MonitoredSEIR(), 
    mng_post, 
    params_post1, 
    'seir_mobility_post.json',
    'SEIR simulation with Mobility Network (Post)'
))

#### SEIR_Q on Mobility

In [148]:
params_pre2 = {**params_mobility_pre, **params_seir, **params_monitor}
params_pre2[SEIRQ.P_INFECT_SYMPTOMATIC] = P_INFECT_MOBILITY
params_pre2[SEIRQ.P_INFECT_ASYMPTOMATIC] = P_INFECT_MOBILITY / 2
params_pre2[SEIRQ.P_QUARANTINE] = _P_QUARANTINE

params_post2 = {**params_mobility_post, **params_seir, **params_monitor}
params_post2[SEIRQ.P_INFECT_SYMPTOMATIC] = P_INFECT_MOBILITY
params_post2[SEIRQ.P_INFECT_ASYMPTOMATIC] = P_INFECT_MOBILITY / 2
params_post2[SEIRQ.P_QUARANTINE] = _P_QUARANTINE

In [149]:
expms.append(lambda: run_stochastic_dynamics_lab(
    MonitoredSEIRQ(), 
    mng_pre, 
    params_pre2, 
    'seirq_mobility_pre.json',
    'SEIR_Q simulation with Mobility Network (Pre)',
))

expms.append(lambda: run_stochastic_dynamics_lab(
    MonitoredSEIRQ(), 
    mng_post, 
    params_post2, 
    'seirq_mobility_post.json',
    'SEIR_Q simulation with Mobility Network (Post)'
))

#### SEIR on PLC

In [150]:
params_pre3 = {**params_plc_pre, **params_seir, **params_monitor}
params_pre3[SEIR.P_INFECT_SYMPTOMATIC] = P_INFECT_PLC
params_pre3[SEIR.P_INFECT_ASYMPTOMATIC] = P_INFECT_PLC / 2

params_post3 = {**params_plc_post, **params_seir, **params_monitor}
params_post3[SEIR.P_INFECT_SYMPTOMATIC] = P_INFECT_PLC
params_post3[SEIR.P_INFECT_ASYMPTOMATIC] = P_INFECT_PLC / 2

In [151]:
expms.append(lambda: run_stochastic_dynamics_lab(
    MonitoredSEIR(), 
    PLCNetwork(), 
    params_pre3, 
    'seir_plc_pre.json',
    'SEIR simulation with PLC Network (Pre)',
))

expms.append(lambda: run_stochastic_dynamics_lab(
    MonitoredSEIR(), 
    PLCNetwork(), 
    params_post3, 
    'seir_plc_post.json',
    'SEIR simulation with PLC Network (Post)'
))

#### SEIR_Q on PLC

In [152]:
params_pre4 = {**params_plc_pre, **params_seir, **params_monitor}
params_pre4[SEIRQ.P_INFECT_SYMPTOMATIC] = P_INFECT_PLC
params_pre4[SEIRQ.P_INFECT_ASYMPTOMATIC] = P_INFECT_PLC / 2
params_pre4[SEIRQ.P_QUARANTINE] = _P_QUARANTINE

params_post4 = {**params_plc_post, **params_seir, **params_monitor}
params_post4[SEIRQ.P_INFECT_SYMPTOMATIC] = P_INFECT_PLC
params_post4[SEIRQ.P_INFECT_ASYMPTOMATIC] = P_INFECT_PLC / 2
params_post4[SEIRQ.P_QUARANTINE] = _P_QUARANTINE

In [153]:
expms.append(lambda: run_stochastic_dynamics_lab(
    MonitoredSEIRQ(), 
    PLCNetwork(), 
    params_pre4, 
    'seirq_plc_pre.json',
    'SEIR_Q simulation with PLC Network (Pre)',
))

expms.append(lambda: run_stochastic_dynamics_lab(
    MonitoredSEIRQ(), 
    PLCNetwork(), 
    params_post4, 
    'seirq_plc_post.json',
    'SEIR_Q simulation with PLC Network (Post)'
))

#### SEIR on Distanced

In [154]:
params_pre5 = {**params_distanced_pre, **params_seir, **params_monitor}
params_pre5[SEIR.P_INFECT_SYMPTOMATIC] = P_INFECT_DISTANCED
params_pre5[SEIR.P_INFECT_ASYMPTOMATIC] = P_INFECT_DISTANCED / 2

params_post5 = {**params_distanced_post, **params_seir, **params_monitor}
params_post5[SEIR.P_INFECT_SYMPTOMATIC] = P_INFECT_DISTANCED
params_post5[SEIR.P_INFECT_ASYMPTOMATIC] = P_INFECT_DISTANCED / 2

In [155]:
expms.append(lambda: run_stochastic_dynamics_lab(
    MonitoredSEIR(), 
    distanced_pre, 
    params_pre5, 
    'seir_distanced_pre.json',
    'SEIR simulation with Distanced Network (Pre)',
))

expms.append(lambda: run_stochastic_dynamics_lab(
    MonitoredSEIR(), 
    distanced_post, 
    params_post5, 
    'seir_distanced_post.json',
    'SEIR simulation with Distanced Network (Post)'
))

#### SEIR_Q on Distanced

In [156]:
params_pre6 = {**params_distanced_pre, **params_seir, **params_monitor}
params_pre6[SEIRQ.P_INFECT_SYMPTOMATIC] = P_INFECT_DISTANCED
params_pre6[SEIRQ.P_INFECT_ASYMPTOMATIC] = P_INFECT_DISTANCED / 2
params_pre6[SEIRQ.P_QUARANTINE] = _P_QUARANTINE

params_post6 = {**params_distanced_post, **params_seir, **params_monitor}
params_post6[SEIRQ.P_INFECT_SYMPTOMATIC] = P_INFECT_DISTANCED
params_post6[SEIRQ.P_INFECT_ASYMPTOMATIC] = P_INFECT_DISTANCED / 2
params_post6[SEIRQ.P_QUARANTINE] = _P_QUARANTINE

In [157]:
expms.append(lambda: run_stochastic_dynamics_lab(
    MonitoredSEIRQ(), 
    distanced_pre, 
    params_pre6, 
    'seirq_distanced_pre.json',
    'SEIR_Q simulation with Distanced Network (Pre)',
))

expms.append(lambda: run_stochastic_dynamics_lab(
    MonitoredSEIRQ(), 
    distanced_post, 
    params_post6, 
    'seirq_distanced_post.json',
    'SEIR_Q simulation with Distanced Network (Post)'
))

### SEIVR & SEIVR_Q

#### SEIVR on Mobility

In [260]:
params_pre7 = {**params_mobility_pre, **params_seivr, **params_monitor}
params_pre7[SEIVR.P_INFECT_SYMPTOMATIC] = P_INFECT_MOBILITY
params_pre7[SEIVR.P_INFECT_ASYMPTOMATIC] = P_INFECT_MOBILITY / 2
params_pre7[SEIVR.P_VACCINATED] = _P_VACCINATED
params_pre7[SEIVR.P_VACCINATED_INITIAL] = _P_VACCINATED_INITAL
params_pre7[SEIVR.VACCINE_RRR] = _RRR

params_post7 = {**params_mobility_post, **params_seivr, **params_monitor}
params_post7[SEIVR.P_INFECT_SYMPTOMATIC] = P_INFECT_MOBILITY
params_post7[SEIVR.P_INFECT_ASYMPTOMATIC] = P_INFECT_MOBILITY / 2
params_post7[SEIVR.P_VACCINATED] = _P_VACCINATED
params_post7[SEIVR.P_VACCINATED_INITIAL] = _P_VACCINATED_INITAL
params_post7[SEIVR.VACCINE_RRR] = _RRR

In [261]:
expms.append(lambda: run_stochastic_dynamics_lab(
    MonitoredSEIVR(), 
    mng_pre, 
    params_pre7, 
    'seivr_mobility_pre.json',
    'SEIVR simulation with Mobility Network (Pre)',
))

In [ ]:
expms.append(lambda: run_stochastic_dynamics_lab(
    MonitoredSEIVR(), 
    mng_post, 
    params_post7, 
    'seivr_mobility_post.json',
    'SEIVR simulation with Mobility Network (Post)'
))

#### SEIVR_Q on Mobility

In [246]:
params_pre8 = {**params_mobility_pre, **params_seivr, **params_monitor}
params_pre8[SEIVRQ.P_INFECT_SYMPTOMATIC] = P_INFECT_MOBILITY
params_pre8[SEIVRQ.P_INFECT_ASYMPTOMATIC] = P_INFECT_MOBILITY / 2
params_pre8[SEIVRQ.P_VACCINATED] = _P_VACCINATED
params_pre8[SEIVRQ.P_VACCINATED_INITIAL] = _P_VACCINATED_INITAL
params_pre8[SEIVRQ.VACCINE_RRR] = _RRR
params_pre8[SEIVRQ.P_QUARANTINE] = _P_QUARANTINE

params_post8 = {**params_mobility_post, **params_seivr, **params_monitor}
params_post8[SEIVRQ.P_INFECT_SYMPTOMATIC] = P_INFECT_MOBILITY
params_post8[SEIVRQ.P_INFECT_ASYMPTOMATIC] = P_INFECT_MOBILITY / 2
params_post8[SEIVRQ.P_VACCINATED] = _P_VACCINATED
params_post8[SEIVRQ.P_VACCINATED_INITIAL] = _P_VACCINATED_INITAL
params_post8[SEIVRQ.VACCINE_RRR] = _RRR
params_post8[SEIVRQ.P_QUARANTINE] = _P_QUARANTINE

In [247]:
expms.append(lambda: run_stochastic_dynamics_lab(
    MonitoredSEIVRQ(), 
    mng_pre, 
    params_pre8, 
    'seivrq_mobility_pre.json',
    'SEIVR_Q simulation with Mobility Network (Pre)',
))

expms.append(lambda: run_stochastic_dynamics_lab(
    MonitoredSEIVRQ(), 
    mng_post, 
    params_post8, 
    'seivrq_mobility_post.json',
    'SEIVR_Q simulation with Mobility Network (Post)'
))

#### SEIVR on PLC

In [212]:
params_pre9 = {**params_plc_pre, **params_seivr, **params_monitor}
params_pre9[SEIVR.P_INFECT_SYMPTOMATIC] = P_INFECT_PLC
params_pre9[SEIVR.P_INFECT_ASYMPTOMATIC] = P_INFECT_PLC / 2
params_pre9[SEIVR.P_VACCINATED] = _P_VACCINATED
params_pre9[SEIVR.P_VACCINATED_INITIAL] = _P_VACCINATED_INITAL
params_pre9[SEIVR.VACCINE_RRR] = _RRR

params_post9 = {**params_plc_post, **params_seivr, **params_monitor}
params_post9[SEIVR.P_INFECT_SYMPTOMATIC] = P_INFECT_PLC
params_post9[SEIVR.P_INFECT_ASYMPTOMATIC] = P_INFECT_PLC / 2
params_post9[SEIVR.P_VACCINATED] = _P_VACCINATED
params_post9[SEIVR.P_VACCINATED_INITIAL] = _P_VACCINATED_INITAL
params_post9[SEIVR.VACCINE_RRR] = _RRR

In [213]:
expms.append(lambda: run_stochastic_dynamics_lab(
    MonitoredSEIVR(), 
    PLCNetwork(), 
    params_pre9, 
    'seivr_plc_pre.json',
    'SEIVR simulation with PLC Network (Pre)',
))

expms.append(lambda: run_stochastic_dynamics_lab(
    MonitoredSEIVR(), 
    PLCNetwork(), 
    params_post9, 
    'seivr_plc_post.json',
    'SEIVR simulation with PLC Network (Post)'
))

#### SEIVR_Q on PLC

In [248]:
params_pre10 = {**params_plc_pre, **params_seivr, **params_monitor}
params_pre10[SEIVRQ.P_INFECT_SYMPTOMATIC] = P_INFECT_PLC
params_pre10[SEIVRQ.P_INFECT_ASYMPTOMATIC] = P_INFECT_PLC / 2
params_pre10[SEIVRQ.P_VACCINATED] = _P_VACCINATED
params_pre10[SEIVRQ.P_VACCINATED_INITIAL] = _P_VACCINATED_INITAL
params_pre10[SEIVRQ.VACCINE_RRR] = _RRR
params_pre10[SEIVRQ.P_QUARANTINE] = _P_QUARANTINE

params_post10 = {**params_plc_post, **params_seivr, **params_monitor}
params_post10[SEIVRQ.P_INFECT_SYMPTOMATIC] = P_INFECT_PLC
params_post10[SEIVRQ.P_INFECT_ASYMPTOMATIC] = P_INFECT_PLC / 2
params_post10[SEIVRQ.P_VACCINATED] = _P_VACCINATED
params_post10[SEIVRQ.P_VACCINATED_INITIAL] = _P_VACCINATED_INITAL
params_post10[SEIVRQ.VACCINE_RRR] = _RRR
params_post10[SEIVRQ.P_QUARANTINE] = _P_QUARANTINE

In [249]:
expms.append(lambda: run_stochastic_dynamics_lab(
    MonitoredSEIVRQ(), 
    PLCNetwork(), 
    params_pre10, 
    'seivrq_plc_pre.json',
    'SEIVR_Q simulation with PLC Network (Pre)',
))

expms.append(lambda: run_stochastic_dynamics_lab(
    MonitoredSEIVRQ(), 
    PLCNetwork(), 
    params_post10, 
    'seivrq_plc_post.json',
    'SEIVR_Q simulation with PLC Network (Post)'
))

#### SEIVR on Distanced

In [216]:
params_pre11 = {**params_distanced_pre, **params_seivr, **params_monitor}
params_pre11[SEIVR.P_INFECT_SYMPTOMATIC] = P_INFECT_DISTANCED
params_pre11[SEIVR.P_INFECT_ASYMPTOMATIC] = P_INFECT_DISTANCED / 2
params_pre11[SEIVR.P_VACCINATED] = _P_VACCINATED
params_pre11[SEIVR.P_VACCINATED_INITIAL] = _P_VACCINATED_INITAL
params_pre11[SEIVR.VACCINE_RRR] = _RRR

params_post11 = {**params_distanced_post, **params_seivr, **params_monitor}
params_post11[SEIVR.P_INFECT_SYMPTOMATIC] = P_INFECT_DISTANCED
params_post11[SEIVR.P_INFECT_ASYMPTOMATIC] = P_INFECT_DISTANCED / 2
params_post11[SEIVR.P_VACCINATED] = _P_VACCINATED
params_post11[SEIVR.P_VACCINATED_INITIAL] = _P_VACCINATED_INITAL
params_post11[SEIVR.VACCINE_RRR] = _RRR

In [217]:
expms.append(lambda: run_stochastic_dynamics_lab(
    MonitoredSEIVR(), 
    distanced_pre, 
    params_pre11, 
    'seivr_distanced_pre.json',
    'SEIVR simulation with Distanced Network (Pre)',
))

expms.append(lambda: run_stochastic_dynamics_lab(
    MonitoredSEIVR(), 
    distanced_post, 
    params_post11, 
    'seivr_distanced_post.json',
    'SEIVR simulation with Distanced Network (Post)'
))

#### SEIVR_Q on Distanced

In [236]:
params_pre12 = {**params_distanced_pre, **params_seivr, **params_monitor}
params_pre12[SEIVRQ.P_INFECT_SYMPTOMATIC] = P_INFECT_DISTANCED
params_pre12[SEIVRQ.P_INFECT_ASYMPTOMATIC] = P_INFECT_DISTANCED / 2
params_pre12[SEIVRQ.P_VACCINATED] = _P_VACCINATED
params_pre12[SEIVRQ.P_VACCINATED_INITIAL] = _P_VACCINATED_INITAL
params_pre12[SEIVRQ.VACCINE_RRR] = _RRR
params_pre12[SEIVRQ.P_QUARANTINE] = _P_QUARANTINE

params_post12 = {**params_distanced_post, **params_seivr, **params_monitor}
params_post12[SEIVRQ.P_INFECT_SYMPTOMATIC] = P_INFECT_DISTANCED
params_post12[SEIVRQ.P_INFECT_ASYMPTOMATIC] = P_INFECT_DISTANCED / 2
params_post12[SEIVRQ.P_VACCINATED] = _P_VACCINATED
params_post12[SEIVRQ.P_VACCINATED_INITIAL] = _P_VACCINATED_INITAL
params_post12[SEIVRQ.VACCINE_RRR] = _RRR
params_post12[SEIVRQ.P_QUARANTINE] = _P_QUARANTINE

In [237]:
expms.append(lambda: run_stochastic_dynamics_lab(
    MonitoredSEIVRQ(), 
    distanced_pre, 
    params_pre12, 
    'seivrq_distanced_pre.json',
    'SEIVR_Q simulation with Distanced Network (Pre)',
))

expms.append(lambda: run_stochastic_dynamics_lab(
    MonitoredSEIVRQ(), 
    distanced_post, 
    params_post12, 
    'seivrq_distanced_post.json',
    'SEIVR_Q simulation with Distanced Network (Post)'
))

## Run experiments with multiprocessing...


In [251]:
# %%time
# num_cores = min(len(expms), cpu_count()-1, 12)
# print(f"Running {num_cores} jobs on {cpu_count()} available cores...")
# print("Start time %s" % dt.now())
# with Parallel(n_jobs=num_cores) as processes:
#     procs = processes(delayed(e)() for e in expms)
# print("End time %s" % dt.now())

for expm in expms:
    print("Start time: %s" % dt.now())
    expm()
    print("End time: %s" % dt.now())

Running 5 jobs on 96 available cores...
Start time 2021-07-16 20:25:57.541047


KeyboardInterrupt: 